In [ ]:
from openai import OpenAI

openai = OpenAI(
    api_key="bIqaNDIFG0wrNSOPmlbIPzqIinK5sTvI",
    base_url="https://api.deepinfra.com/v1/openai",
)

stream = False # or False

chat_completion = openai.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[{"role": "user", "content": "Tell me a long joke."}],
    stream=stream,
    temperature=1,
)

if stream:
    for event in chat_completion:
        if event.choices[0].finish_reason:
            print(event.choices[0].finish_reason,
                  event.usage['prompt_tokens'],
                  event.usage['completion_tokens'])
        else:
            print(event.choices[0].delta.content)
else:
    print(chat_completion.choices[0].message.content)

In [ ]:
from recorder import MASDataset

dataset = MASDataset.load_pickle("/home/hanwen/workspace/2025/MASLab/results/Meta-Llama-3-8B-Instruct/MMLU3/llm_debate/lcul7awf_infer.pkl")


In [ ]:
dataset.questions[0].final_response.generation_list

In [ ]:
import json

with open("/home/hanwen/workspace/2025/MASLab/datasets/data/MMLU.json", "r") as f:
    data = json.load(f)

for i in range(5):
    subset = data[i*100:(i+1)*100]
    with open(f"/home/hanwen/workspace/2025/MASLab/datasets/data/MMLU_{i}.json", "w") as f:
        json.dump(subset, f, indent=4)

In [5]:
from concurrent.futures import ThreadPoolExecutor
import time
from tqdm import tqdm
import threading

# Shared progress bar with thread lock


def task(n, pbar, pbar_lock):
    for i in range(100):
        time.sleep(0.1)
        
        # Update shared progress bar safely
        with pbar_lock:
            pbar.update(1)
    
    return f"Task {n} completed"

def worker():
    # Calculate total iterations across all tasks
    total_iterations = 5 * 100  # 5 tasks × 100 iterations each
    pbar = tqdm(total=total_iterations, desc="Overall progress")
    pbar_lock = threading.Lock()

    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(task, i, pbar, pbar_lock) for i in range(5)]
        
        for future in futures:
            result = future.result()
            print(result)

    pbar.close()
    
worker()

Overall progress: 100%|██████████| 500/500 [00:10<00:00, 49.82it/s]

Task 0 completed
Task 1 completed
Task 2 completed
Task 3 completed
Task 4 completed
